# AxiSEM3D Example 4: Simple 3D Shapes 


* [Overview](#overview)
* [Running this notebook...](#important)
* [Example 1: Simple spherical blob](#example_1)
    - [1.0 Overview](#example_1_0)
    - [1.1 Creating the sphere](#example_1_1) 
    - [1.2 Running the simulation](#example_1_2) 
    - [1.3 Analysing the output](#example_1_3) 
    - [1.4 Some notes on domain dimensions](#example_1_4) 
* [Example 2: Multipled distributed blobs](#example_2)
* [Example 3: A plume in a global domain](#example_3)
    - [3.0 Overview](#example_3_0)
    - [3.1 Making our 1D mesh](#example_3_1) 
    - [3.2 Creating our 3D plume](#example_3_2) 
    - [3.3 Checking 3D model in paraview](#example_3_3) 
    - [3.4 Adding 3D model to the simulation](#example_3_4) 
    - [3.5 Running the simulation](#example_3_5) 
* [Example 4: Release paper example](#example_4)

* [Appendix: Viewing .nc files in Paraview](#appendix)



## Overview: <a class="anchor" id="overview"></a>
This example is designed to highlight the flexibility of AxiSEM3D to incorporating different geometric shapes into both cartesian and global simulations. Introduction of sphere's or ellipsoids, for example, may be used to study wave scattering, while cylinders and slabs may be used to in studies regarding tectonic structures like plumes or subducting material. 

Here I provide four examples: 
1) Single sphere in small 3D cartesian grid  
2) Multiple distributed spheres in a 3D cartesian grid 
3) A plume head and tail in a global simulation 
4) An example used in the release paper for AxiSEM3D (DOI: ... )

*Please note that in Examples 1-3 I use a very low $n_r$ number (either 1 or 5). This makes the simulations much faster but NOT accurate. In these cases it doesnt really matter but if you want a realistic simulation then high $n_r$ is the way to go!*

If you have any questions about the examples or find any bugs, please either open an issue on the AxiSEM3D repository or drop me an email: weaton@princeton.edu 



## Running this notebook: <a class="anchor" id="important"></a>
This example relies on pre-processing scripts called [AxiSEM3D Shapes](https://github.com/williameaton/axisem3d-shapes). For convenience a version of the source code needed for these examples is shipped with AxiSEM3D and is in the ```src``` directory of this example. However a newer version may be available so check the Github repository.

**To use this notebook you will need to run the cell below**

In [ ]:
# NEED TO FIX THE SETUP FOR USERS 
from IPython.display import Image
from IPython.core.display import HTML
from IPython.display import Video
import numpy as np
import sys 
sys.path.append('./src')

## Example 1: Simple ellipsoidal blob<a class="anchor" id="example_1"></a>

### 1.0) Overview<a class="anchor" id="example_1_0"></a>

In this example we will create a cartesian domain of dimensions 20 km x 10 km x 12 km and add an ellipsoid of radii 6 km x 6 km x 1 km at a distance of 13 km along the positive x-axis. We will then have a source at the centre of the domain and study the resulting wavefield. 


### 1.1) Creating the sphere <a class="anchor" id="example_1_1"></a>

In [11]:
# Import the relevant classes
from model import Model 
from ellipsoid import Ellipsoid 
from injector import Injector

# Create a model object with the dimensions of the domain. 
# Note that in this example where we are injecting just one blob, we could make the domain dimensions simply just a box that
# fits around our sphere of interest - this will substantially reduce the size of the overall .nc file and is therefore 
# useful (and sometimes imperetive) for single shapes in large domains. Here however, the domain is small so it doesnt really
# matter. I discuss this idea a bit more in Example 2. 
m = Model(type = "cartesian",
          x_lim = [-10000, 20000], 
          y_lim = [-6000, 6000], 
          z_lim = [0, 17000], 
          elements_per_wavelength = 3, 
          dominant_freq = 3,
          min_velocity = 900, 
          )


# Create an ellipse: 
ellpsoid = Ellipsoid(model = m, 
                    vp  = -0.2,
                    vs  = -1,
                    rho = -0.2,
                    dim = [3000, 3000, 3000, 0, 0, 1])


# Create an injector object for the model 
i = Injector(m)

# Add an object at the centre of the domain using injector 
i.addObj(ellpsoid, location = [13000, 0, 8500])

# Write to netcdf file
m.writeNetCDF("./example_input_cartesian/example_sphere_cartesian.nc", paraview=True)

Generated ellipsoid.
Writing cartesian model...
Data written to file  ./example_input_cartesian/example_sphere_cartesian.nc


I would recommend always checking your 3D model with paraview before using it in AxiSEM-3D to ensure it is what you expect. A very brief guide on opening these models in Paraview is found at the bottom of this notebook. 


### 1.2) Running the simulation <a class="anchor" id="example_1_2"></a>

#### Creating the mesh: 
A .bm file (background model, not bitmap!) file is supplied. Here we are using a homogenous background model for simplicity. We can create the exodus mesh by running the command stored in ```gen_mesh.sh```: 

```$ bash gen_mesh.sh```

should do the trick! 

All of the other inparam files are provided. Make sure all the inparam YAML files, the exodus mesh and the .nc file are all within the ```input``` directory. Then, using the terminal, move to the ```build``` directory and run the AxiSEM3D executable: 

```$ cd .. ``` \
```$ mpirun ./axisem3d``` 


### 1.3) Analysing the output <a class="anchor" id="example_1_3"></a>

**STILL NEED TO ATTACH VIDEO HERE**


### 1.4) Some notes on domain dimensions. <a class="anchor" id="example_1_4"></a>

You may have noticed that in the AxiSEM_shapes model the dimensions are set as 0, 20000 [m] in the x direction, but in the script for generating the 1D mesh the x dimension is 10000. To explain this, I refer to the diagram below 

<img src="./images/domain_diagram.png" width=800 height=300/>

The 1D mesh shown in (a) is rotated around around the $\hat{Z}$ axis to form a cylinder. The $\hat{Z}$ axis is found at $x=0, y=0$. Hence the cylinder is defined from [-10000, 10000]. Inside AxiSEM_shapes this conversion is actually performed so that 0 to 20 km is converted to [-10 km, 10 km]. The domain created in the script is shown by the purple box in (a). This placement of 3D models relative to the axis that you choose for rotation is important.  



## Example 2: Multiple distributed blobs<a class="anchor" id="example_2"></a>

This example will be brief. It is simply to highlight the options for putting 3D models into AxiSEM3D. 

Imagine we want to add in 5 spheres of the same size, but at different locations. We have two options. The first is to create a single .nc file that holds all 5 spheres. The second option is to to create 5 individual .nc files. You can imagine the later option could be done by looping example 1 with 5 different locations. If we want to keep all 5 spheres in a single .nc file then we could edit the script in Example 1 with something like this: 

```
locations = [[17000, 0, 8500], [-5000, 2000, 8500], [2000, 4000, 2000], [10000, -4000, 0], [0, 0, 0]] 

for loc in locations: 
    i.addObj(ell, location = loc)
```

*So which option is best?*

Remember that your model domain will need to be large enough to encapsulate all the shapes you want to include. This means that if you want to put all your sphere's in one .nc file then it will need to be a rather large file (as the 3D arrays will get very large very fast!). 

Alternatively, if you have 5 .nc files then you need to put 5 different model entries into your inparam.model.yaml file. This may be prone to mistakes and is laborious. 

The .nc files are only loaded by a single node, even when using a cluster. This means that you are somewhat limited by the file size that can be loaded in serial by axisem3d. Hence, if your .nc file is very large then its probably best to have multiple, smaller .nc files. 

Note that the axisem3d-shapes code does come with a function that injects a shape at equal spacing throughout your model (```inj.spacedSpheres()```) if, for example, you want to inject 1000 spheres which would be very inefficient to do as seperate .nc files. 

## Example 3: A plume in a global domain <a class="anchor" id="example_3"></a>

### 3.0) Overview<a class="anchor" id="example_3_0"></a>
Here we will make a plume using a cylinder and an ellipsoid, then inject it into our model and look at the results. 


### 3.1) Making our 1D mesh <a class="anchor" id="example_3_1"></a>
Although this is a global, Earth-like simulation, we will use a homogenous domain with a radius of 4000 km. The reason for this is to reduce the computational demands (no need to simulate the extra 2000 km!). If you want to see how to run an Earth-like simulation with a realistic model like PREM then checkout my Example 4 [here](https://github.com/AxiSEMunity/AxiSEM3D-further-examples/tree/main/examples). The homogenous background model is simply so that the effects of the plume are easier to distinguish, but any 1D background model of 4000km radius would be fine! 

A file called homogenous_4000km.bm is provided. To generate we can run the command 

```$ python -m salvus_mesh_lite.interface AxiSEM --basic.model homo.bm --basic.period 8 --output_file homogenous.e```

from within the ```input``` directory. This generates a mesh called ```homogenous.e```. 

### 3.2) Creating our 3D plume <a class="anchor" id="example_3_2"></a>

The plume will have its tail axis at [0, 0] in coordinates and will stretch from a depth of 1000 km to 3000 km. The plume head is a sphere (ellipsoid) of radius 500 km centred at a depth of 1200 km. The Vp, Vs and density perturbation will all be - 10 %. 
Now we will generate a 3D model to be incorporated into the simulation: 

In [4]:
import numpy as np
import scipy.ndimage as sci
from model import Model
from ellipsoid import Ellipsoid
from cylinder import *
from injector import *
import netCDF4 as nc

# We dont need to make a model that spans the whole domain, just the part we are interested in injecting a plume in: 
radius = 4000000
perturb = -0.1
lat_lim = [-20, 20]
long_lim = [-20, 20]
depth_lim = [0, radius]

# Set locations for shapes:
ell_loc = [0,0, 1200000]
cyl_loc = [0,0, 2000000]

# Create our global model: 
glob_m = Model("spherical", lat_lim, long_lim, depth_lim, elements_per_wavelength=1, dominant_freq=1, min_velocity=10000, oversaturation=1, a=radius)

# Create cylinder: 
cylinder = Cylinder(model=glob_m, vp=perturb, vs=perturb, rho=perturb, dim=[2000000, 100000, 0, 0, 1], loc=cyl_loc, major_axis='Z')

# Create ellipse: 
ellipse = Ellipsoid(model=glob_m, vp=perturb, vs=perturb, rho=perturb, dim=[500000, 500000, 500000, np.pi/2, 0, 1], loc=ell_loc)

# Create injector object and inject
i = Injector(glob_m)
i.addObj(cylinder, location=cyl_loc, overwrite=True)
i.addObj(ellipse, location=ell_loc, overwrite=True)

# Gaussian filter to make the plume boundaries a bit less harsh 
sigma = 1
glob_m.bm_rho =  sci.gaussian_filter(input=glob_m.bm_rho, sigma=sigma)
glob_m.bm_vp =  sci.gaussian_filter(input=glob_m.bm_vp, sigma=sigma)
glob_m.bm_vs =  sci.gaussian_filter(input=glob_m.bm_vs, sigma=sigma)

# Write to NetCDF file 
out_path = f"./example_single_plume/plume_{sigma}"
glob_m.writeNetCDF(f"{out_path}.nc")
glob_m.writeNetCDF(f"{out_path}_visual.nc", paraview=True)




Major axis: Z
Generated cylinder.
Generated ellipsoid.
Writing spherical model...
PARAVIEW FLAG = FALSE: MODEL OKAY FOR SIMULATION
Data written to file  ./example_single_plume/plume_1.nc
# ___________________________________________________________________________________
    - ./example_single_plume/plume_1:
        activated: true
        class_name: StructuredGridV3D
        nc_data_file: ./example_single_plume/plume_1.nc
        coordinates:
            horizontal: LATITUDE_LONGITUDE
            vertical: DEPTH
            ellipticity: FILL THIS IN - true/false
            depth_below_solid_surface: FILL THIS IN - true/false
            nc_variables: [lat, lon, depth]
            data_rank: [1, 2, 0]
            length_unit: m
            angle_unit: degree
            undulated_geometry: false
            whole_element_inplane: false
        properties:
            - VP:
                nc_var: vp
                factor: 1
                reference_kind: FILL THIS IN - ABS/REF1D/R

### 3.3) Checking 3D model in paraview<a class="anchor" id="example_3_3"></a>
Note that above we wrote out two versions of the model, called ```plume.nc``` and ```plume_visual.nc```, the latter of which has a flag set to ```paraview=True```. When this flag is on the depth coordinates are switched to radius values. The reason for this is that paraview takes the coordinates as radii and so the model appears wrong when you try to visualise it. Below I show the same model but saved with the flag switched on/off. 

**NOTE HOWEVER THAT THE MODEL IN WHICH ```paraview=False``` IS WHAT WE WILL USE FOR RUNNING THE SIMULATION.**

<img src="./images/paraview_flag.png" width=600 height=300 />

### 3.4) Adding 3D model to the simulation: <a class="anchor" id="example_3_4"></a>
So we now have our plume in a .nc file, but now we need to incorporate it into the simulation. You'll notice that the ```glob_m.writeNetCDF("plume.nc")``` command caused a lot of YAML-like stuff to be printed between the #------ lines. This can be copied and pasted into the ```inparam.model.yaml``` file, but I've already done it for you! 

**Make sure that the name of the .nc file and any file path (if not in the same folder as inparam.model.yaml) are correct!**

**There are some sections you still need to fill in though!** For example, here we need to change:

 ```reference_kind: REF1D``` - **this is important as our .nc holds values of 0. and -0.1 which are perturbation percentages relative to the background model.** \
 ```depth_below_solid_surface: true``` - this is arbitrary as we have no fluid regions. \
 ```ellipticity: false``` - again not really important for this simulation. 


### 3.5) Running the simulation: <a class="anchor" id="example_3_5"></a>
Now we have all our files and we can run the simulation using the inparam scripts provided. Depending on the period you choose for your background model this can take anywhere between seconds and hours. Using a period of 8 seconds it took about 2 hours on my mac or about 6 minutes on Princeton's Tiger cluster with 2 nodes of 40 cores.

Also in this directory is the ```gen_movie.py``` which can be run when the simulation is finished to generate a wavefield animation (though you will need to move ```gen_movie.py``` to the ```build``` directory). 

In the cell below is a video where I compare the output of this simulation with a simulation with no plume (if you really want to do this yourself you can set the plume model to ```activated: false``` in ```inparam.model.yaml``` and rerun the whole thing), to produce the homogenous results. The left hemisphere is the homogenous simulation: 

In [3]:
Video("./images/plume.mov", height=500)

## Example 4: Release paper <a class="anchor" id="example_4"></a>

This example reproduces the results of the AxiSEM3D release paper (Nissen-Meyer et al, in prep), Section 3.1.4 entitled *Wave scattering: arbitrary 3D heterogeneity*. Everything related to this example may be found in the `example_release_paper` directory. Steps to reproduce Figure ... are listed below: 

1. To generate the 3D `NetCDF` model, you should run the python script `generate_3D_model.py`, e.g.
    ```bash 
        $ cd example_release_paper 
        $ python3 -m generate_3D_model
    ```
    *Note here that although this script can generate a new, random model of ellipsoids, it is set here to read values from the file `randomblobs` such that the `NetCDF` file utilised in the release paper example can be reproduced exactly. See comments in the file for further details*

2. All input files, apart from the 3D model (.nc file) are provided in `example_release_paper/inputs`. These files should be added to your `input` directory of the AxiSEM3D build before simulation. 
3. Run simulation. Depending on the resources available it may take a few hours! 
4. Download the `output/Stations/Station_1/` data into the `example_release_paper/datas` directory and run the plotting script.



## Appendix: Viewing your NetCDF file in paraview <a class="anchor" id="appendix"></a>

### Loading into paraview 
Both the exodus meshes (.e) used for the 1D model, and the NetCDF files (.nc) can be viewed in paraview, but I will talk about .nc files here. 

To open your NetCDF file load Paraview and right-click somewhere in the ```Pipeline Browser``` section, then select ```Open```. 


<img src="./images/netcdf/open_paraview.png" width=800 height=500 />
<img src="./images/netcdf/open_file1.png" width=800 height=500 />


Double click on the .nc file you want to load. When it asks how to open the data, I use the second option ```NetCDF Reader```. 


<img src="./images/netcdf/open_file2.png" width=800 height=500 />
<img src="./images/netcdf/open_file3.png" width=800 height=500 />

<img src="./images/netcdf/open_data_with.png" width=800 height=500 />

In the properties window you can change certain parameters if you like, and then click ```Apply```. to apply the parameters. By default it will show the .nc data as an ```Outline``` so you will just see the outline of your model file. 

<img src="./images/netcdf/apply.png" width=200 height=250 />

For the examples in this tutorial, we really care about specific shapes that are in our 3D model. To isolate these shapes and look at them we can use the ```Threshold``` feature. Click on the .nc file in the ```Pipeline Browser``` and then click on the Threshold icon just above it. 

<img src="./images/netcdf/threshold_button.png" width=800 height=500 />



We can now isolate a part of our model based on its values. In this case we can isolate a shape based on its density by selecting ```Scalar``` to be `rho` in the ```Properties``` window. By default the threshold is set to the entire range spanned by the model. Select a threshold from -0.2 to -0.1 and click Apply. You will see the isolated sphere of the model. 


<img src="./images/netcdf/threshold_range1.png" width=800 height=500 />
<img src="./images/netcdf/threshold_range2.png" width=800 height=500 />


Further down in the properties window you can use the ```Representation``` and ```Colouring``` options to visualise the value of rho/vp/vs etc in your isolated shape. 


